<a href="https://colab.research.google.com/github/wtkns/-CHAZ/blob/master/birdsong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install build-essential cmake libjack-jackd2-dev libsndfile1-dev libfftw3-dev libxt-dev libavahi-client-dev
!sudo apt-get install git libasound2-dev libicu-dev libreadline6-dev libudev-dev pkg-config libncurses5-dev

!git clone --recursive https://github.com/supercollider/supercollider.git

import os
os.chdir('supercollider')

!git checkout Version-3.12.2

os.makedirs('build')
os.chdir('build')

!cmake -DSC_QT=OFF -DCMAKE_BUILD_TYPE=Release -DNATIVE=ON -DNO_X11=ON -DSC_EL=no ..
!make -j3
!sudo make install

# if we end up needing sc3-plugins:

# os.chdir('/content')
# !git clone --recursive https://github.com/supercollider/sc3-plugins.git
# os.makedirs('sc3-plugins/build')
# os.chdir('sc3-plugins/build')

# !cmake -DSC_PATH=/content/supercollider -DCMAKE_BUILD_TYPE=Release -DSUPERNOVA=ON ..
# !cmake --build . --config Release
# !sudo cmake --build . --config Release --target install

# !sclang -v
# !echo '"hello world".postln;'>test.scd
# !sclang ./test.scd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libxt-dev is already the newest version (1:1.1.5-1).
libxt-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libsndfile1-dev is already the newest version (1.0.28-4ubuntu0.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libavahi-common-dev libdbus-1-dev libfftw3-bin libfftw3-long3 libfftw3-quad3
  libfftw3-single3 libjack-jackd2-0
Suggested packages:
  libfftw3-doc jackd2
The following packages will be REMOVED:
  libjack-dev libjack0
The following NEW packages will be installed:
  libavahi-client-dev libavahi-common-dev libdbus-1-dev libfftw3-bin
  libfftw3-dev libfftw3-long3 libfftw3-quad3 libfftw3-single3 libjack-jackd2-0
  libjack-

In [8]:
os.chdir('/content')
os.makedirs('output')

In [9]:
%%writefile hansm-bird-nrt.scd
/* hansm-bird-control.pd
http://obiwannabe.co.uk/tutorials/html/tutorial_birds.html
SC port by redFrik 080430 */

  var myPath = "./output/" ++ thisProcess.argv[0] ++ ".wav";
  var argfreq = thisProcess.argv[1].asFloat;
  var argatk = thisProcess.argv[2].asFloat;
  var argdcy = thisProcess.argv[3].asFloat;
  var argfmod1 = thisProcess.argv[4].asFloat;
  var argatkf1 = thisProcess.argv[5].asFloat;
  var argdcyf1 = thisProcess.argv[6].asFloat;
  var argfmod2 = thisProcess.argv[7].asFloat;
  var argatkf2 = thisProcess.argv[8].asFloat;
  var argdcyf2 = thisProcess.argv[9].asFloat;
  var argamod1 = thisProcess.argv[10].asFloat;
  var argatka1 = thisProcess.argv[11].asFloat;
  var argdcya1 = thisProcess.argv[12].asFloat;
  var argamod2 = thisProcess.argv[13].asFloat;
  var argatka2 = thisProcess.argv[14].asFloat;
  var argdcya2 = thisProcess.argv[15].asFloat;

  var server = Server(\nrt,
      options: ServerOptions.new
      .numOutputBusChannels_(2)
  );

  var birdsynth = SynthDef(\birdCall, {|
			out= 0, pan= 0, gate= 1, freq= 0, amp= 0.5, atk= 0.5, dcy= 0.5,
			fmod1= 1, atkf1= 0.5, dcyf1= 0.5,
			fmod2= 1, atkf2= 0.5, dcyf2= 0.5,
			amod1= 1, atka1= 0.5, dcya1= 0.5,
			amod2= 1, atka2= 0.5, dcya2= 0.5|

			var env, freq1, freq2, amp1, amp2, fmod, amod, z;
			env= EnvGen.ar(Env([0, amp, 0], [atk, dcy], -4), gate, timeScale:0.9, doneAction:2);
			freq1= EnvGen.ar(Env([0, fmod1, 0], [atkf1, dcyf1], -4), 1, 3000, 0, 0.9);
			freq2= EnvGen.ar(Env([0, fmod2, 0], [atkf2, dcyf2], -4), 1, 3000, 0, 0.9);
			amp1= EnvGen.ar(Env([0, amod1, 0], [atka1, dcya1], -4), 1, 1, 0, 0.9);
			amp2= EnvGen.ar(Env([0, amod2, 0], [atka2, dcya2], -4), 1, 1, 0, 0.9);
			fmod= SinOsc.ar(freq1, 0, amp1, 1);
			amod= 1-SinOsc.ar(freq2, 0, amp2);
			z= SinOsc.ar(freq*7000+300*fmod, 0, amod);
			Out.ar(out, Pan2.ar(z, pan, env));
	});

  var myScore = Score([
      [0.0, ['/d_recv',birdsynth.asBytes]],
      [0.0, (x = Synth.basicNew(\birdCall, server)).newMsg(
      args: [freq: argfreq, atk: argatk, dcy: argdcy,
      fmod1: argfmod1, atkf1: argatkf1, dcyf1: argdcyf1,
      fmod2: argfmod2, atkf2: argatkf2, dcyf2: argdcyf2,
      amod1: argamod1, atka1: argatka1, dcya1: argdcya1,
      amod2: argamod2, atka2: argatka2, dcya2: argdcya2]
      )],
      [1.0, x.freeMsg]
  ]);

  var myRoutine = Routine.new({
      var cond = Condition.new;
      
      myScore.recordNRT(
        outputFilePath:myPath.standardizePath,
        headerFormat: "WAV",
        sampleFormat: "int16",
        options: server.options,
        action: { cond.unhang }
      );

      cond.hang;	
      "done".postln;	
      0.exit;
      
  }); // end Routine

  myRoutine.play;  // 'play' is a shortcut for scheduling the routine on a clock


Overwriting hansm-bird-nrt.scd


In [42]:
%%writefile example-birds.txt
triple-tailed-tree-troubler 0.387755 0.0204082 0.204082 0.367347 0.571429 0.734694 0.918367 1 0.77551 0.571429 0.367347 0.22449 0.0204082 0.183673 0.44898
speckled-throated-spew 0.183673 0.591837 0.387755 0.0104082 0.530612 0.346939 0.244898 0.55102 0.122449 0.387755 1 0.612245 0.346939 0.816327 0.653061
lesser-spotted-grinchwarbler 0.55102 0.591837 0.387755 0.0716327 0.0204082 0.346939 0.0204082 0.55102 0.122449 0.632653 1 0.612245 0.346939 0.816327 0.653061
long-toed-mudhopper 0.163265 0.22449 0.183673 0.00306122 0.122449 1 0.0612245 1 0.77551 0.979592 0.204082 0.734694 1 0.142857 0.612245
yellow-yiffled-yaffle 0.0204082 0.367347 0.183673 0.0612245 0 1 0.285714 0.22449 0.489796 0.367347 0.387755 0.734694 0.204082 0.428571 0.142857
pointy-beaked-beetlefiend 0.428571 0.204082 0.489796 0.0204082 0.795918 0.591837 0.285714 0.22449 0.489796 0.204082 0.836735 0.734694 0.77551 0.428571 0.142857
african-boojuboolubala 0.306122 0.959184 0.0408163 1 0 0.591837 0.285714 0.22449 0.489796 0.204082 0.836735 0.734694 0.77551 0.428571 0.142857
common-muckoink 0.0204082 0.8 0.0816327 0.0204082 0.001 0.99 0.0204082 0.01 1 1 0.142857 0.734694 1 0.0612245 0.530612

Overwriting example-birds.txt


In [38]:
# for running ffmpeg
import subprocess
import sys

def generate_birdsong(parameter_list):

  sc_command_string = 'sclang ./hansm-bird-nrt.scd '
  sc_command_string += ' '.join(parameter_list)

  result = subprocess.Popen(sc_command_string, stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell=True)
  stdout, stderr = result.communicate()
  
  return (str(stdout))

In [39]:
def parameter_dict_to_list(parameter_dict):
  parameter_list = []
  parameter_list.append(parameter_dict['name'])
  parameter_list.append(parameter_dict['freq'])
  parameter_list.append(parameter_dict['atk'])
  parameter_list.append(parameter_dict['dcy'])
  parameter_list.append(parameter_dict['fmod1'])
  parameter_list.append(parameter_dict['atkf1'])
  parameter_list.append(parameter_dict['dcyf1'])
  parameter_list.append(parameter_dict['fmod2'])
  parameter_list.append(parameter_dict['atkf2'])
  parameter_list.append(parameter_dict['dcyf2'])
  parameter_list.append(parameter_dict['amod1'])
  parameter_list.append(parameter_dict['atka1'])
  parameter_list.append(parameter_dict['dcya1'])
  parameter_list.append(parameter_dict['amod2'])
  parameter_list.append(parameter_dict['atka2'])
  parameter_list.append(parameter_dict['dcya2'])

  return parameter_list

def parameter_list_to_dict(parameter_list):
  parameter_dict = {
    'name'  : str(parameter_list[0]),
    'freq'  : str(parameter_list[1]),
    'atk'   : str(parameter_list[2]),
    'dcy'   : str(parameter_list[3]),
    'fmod1' : str(parameter_list[4]),
    'atkf1' : str(parameter_list[5]),
    'dcyf1' : str(parameter_list[6]),
    'fmod2' : str(parameter_list[7]),
    'atkf2' : str(parameter_list[8]),
    'dcyf2' : str(parameter_list[9]),
    'amod1' : str(parameter_list[10]),
    'atka1' : str(parameter_list[11]),
    'dcya1' : str(parameter_list[12]),
    'amod2' : str(parameter_list[13]),
    'atka2' : str(parameter_list[14]),
    'dcya2' : str(parameter_list[15])
  }

  return parameter_dict

In [40]:
bird_dict = parameter_list_to_dict(['triple-tailed-tree-troubler', 0.387755, 0.0204082, 0.204082, 0.367347, 0.571429, 0.734694, 0.918367, 1, 0.77551, 0.571429, 0.367347, 0.22449, 0.0204082, 0.183673, 0.44898])

print(bird_dict)

bird_list = parameter_dict_to_list(bird_dict)

print(bird_list)

{'name': 'triple-tailed-tree-troubler', 'freq': '0.387755', 'atk': '0.0204082', 'dcy': '0.204082', 'fmod1': '0.367347', 'atkf1': '0.571429', 'dcyf1': '0.734694', 'fmod2': '0.918367', 'atkf2': '1', 'dcyf2': '0.77551', 'amod1': '0.571429', 'atka1': '0.367347', 'dcya1': '0.22449', 'amod2': '0.0204082', 'atka2': '0.183673', 'dcya2': '0.44898'}
['triple-tailed-tree-troubler', '0.387755', '0.0204082', '0.204082', '0.367347', '0.571429', '0.734694', '0.918367', '1', '0.77551', '0.571429', '0.367347', '0.22449', '0.0204082', '0.183673', '0.44898']


In [41]:
generate_birdsong(bird_list)

'b"compiling class library...\\n\\tFound 738 primitives.\\n\\tCompiling directory \'/usr/local/share/SuperCollider/SCClassLibrary\'\\n\\tCompiling directory \'/usr/local/share/SuperCollider/Extensions\'\\n\\tCompiling directory \'/root/.local/share/SuperCollider/Extensions\'\\n\\tnumentries = 679892 / 7365942 = 0.092\\n\\t3973 method selectors, 1854 classes\\n\\tmethod table size 9890224 bytes, big table size 58927536\\n\\tNumber of Symbols 9257\\n\\tByte Code Size 265366\\n\\tcompiled 229 files in 0.19 seconds\\n\\nInfo: 4 methods are currently overwritten by extensions. To see which, execute:\\nMethodOverride.printAll\\n\\ncompile done\\nlocalhost : setting clientID to 0.\\ninternal : setting clientID to 0.\\nCouldn\'t set realtime scheduling priority 1: Operation not permitted\\nClass tree inited in 0.01 seconds\\n\\n\\n*** Welcome to SuperCollider 3.12.2. *** For help type ctrl-c ctrl-h (Emacs) or :SChelp (vim) or ctrl-U (sced/gedit).\\nCouldn\'t set realtime scheduling priority 1:

In [51]:
example_file = open("example-birds.txt", "r")
raw_examples_list = example_file.readlines()
example_list = []
for example in raw_examples_list:
  example_list.append(example.split(" "))

for example in example_list:
  generate_birdsong(example)



In [55]:
import IPython

directory = 'output'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    print(filename)

triple-tailed-tree-troubler.wav
common-muckoink.wav
african-boojuboolubala.wav
long-toed-mudhopper.wav
speckled-throated-spew.wav
lesser-spotted-grinchwarbler.wav
pointy-beaked-beetlefiend.wav
.ipynb_checkpoints
yellow-yiffled-yaffle.wav
